In [1]:
import pandas as pd
import numpy as np

import re

from utils.wordle import Wordle
from utils.reverse_wordle import *


import os

# playing wordle by statistics

In [2]:
# Wordle Game Setting
n_letter = 5

wordle_file = f"./data/common {n_letter}-letter words.csv"

wd = Wordle(wordle_file)
wd.ans

'beebe'

In [3]:
wddf = pd.read_csv(wordle_file)
wddf = wddf.dropna()
wddf

,word,count
0,aaron,7
1,aback,2
2,abbas,2
3,abbey,7
4,abbot,2
...,...,...
4067,ziggy,1
4068,zoned,1
4069,zones,3
4070,zooey,1


# Create Score Matrix

In [4]:
score_file = f"./guess score matrix {n_letter}-letter words.csv"

vocab = list(wddf['word'])

df = get_score_matrix(wordle_file, score_file)

# Play

In [5]:
# if want to fixed a word to compare 2 methods

fixed_ans = True
fixed_word = "perky"
# fixed_word = Wordle(wordle_file).ans

In [6]:
wd = Wordle(wordle_file)
# overide
if fixed_ans:
    wd.ans = fixed_word
    
print("(hidden) answer:", wd.ans)
print("")


# first guess
guess = None
correct = False
possible_ind = range(len(df))

while not(correct):
    # guess turn i
    
    # the first turn pick a word from V0 i.e. vocab
    # pick word from V(i-1), based on previous possible vocab
    if guess == None:
        guess = np.random.choice(vocab, 1)[0]
    else:
        guess = np.random.choice(possible_words, 1)[0]
        
    wd_result = wd.guess(guess)
    guess_result = encode_score(wd_result['score'])
    
    # create new vocab based on guess turn i and score turn i
    possible_ind, possible_words = filter_possible_words(df, possible_ind, guess, guess_result)
    
    correct = (wd_result['score'] == 2).all()
    
    print(f"guess: {wd_result['#guess']} | {wd_result['guessed word']} | {''.join([str(i) for i in wd_result['score']])}")
    print(f"words left: {len(possible_words)}")
    if len(possible_words) < 10:
        print(f"{possible_words}")
    print("")

(hidden) answer: perky

guess: 1 | flake | 00021
words left: 10

guess: 2 | jenks | 02020
words left: 1
['perky']

guess: 3 | perky | 22222
words left: 1
['perky']



# Play by select best word

In [7]:
wd = Wordle(wordle_file)
# overide
if fixed_ans:
    wd.ans = fixed_word

print("(hidden) answer:", wd.ans)
print("")

# first guess
guess = None
correct = False
possible_ind = range(len(df))

while not(correct):
    # guess turn i
    
    # the first turn pick a word from V0 i.e. vocab
    # pick word from V(i-1), based on previous possible vocab
    if guess == None:
        guess = find_best_guess(df, range(len(df)), use_all_word=True)
    else:
        guess = find_best_guess(df, possible_ind, use_all_word=True)
        
    wd_result = wd.guess(guess)
    guess_result = encode_score(wd_result['score'])
    
    # create new vocab based on guess turn i and score turn i
    possible_ind, possible_words = filter_possible_words(df, possible_ind, guess, guess_result)
    
    correct = (wd_result['score'] == 2).all()
    
    print(f"guess: {wd_result['#guess']} | {wd_result['guessed word']} | {''.join([str(i) for i in wd_result['score']])}")
    print(f"words left: {len(possible_words)}")
    if len(possible_words) < 10:
        print(f"{possible_words}")
    print("")

(hidden) answer: perky

saner: expected vocab size = 88.5
guess: 1 | saner | 00011
words left: 131

boite: expected vocab size = 9.2
guess: 2 | boite | 00001
words left: 23

kempe: expected vocab size = 2.8
guess: 3 | kempe | 12010
words left: 1
['perky']

guess: 4 | perky | 22222
words left: 1
['perky']

